In [1]:
from harpy import (
    Session,
    MapTask,
    ReduceTask,
    TransformTask,
    Result,
    TaskSetResults
)

session = Session().create_session()

In [2]:
session.fs.ls("../_example_data/motor_colisions/")

TaskSet ts-7bf2fb62-7915-499e-bca5-0c634a7c07c0: running
TaskGroup tg-f52b1a9c-6051-4206-93b4-19dc1acbd505 made progress
TaskGroup tg-f52b1a9c-6051-4206-93b4-19dc1acbd505 made progress
Task tg-f52b1a9c-6051-4206-93b4-19dc1acbd505-tr-0: queued
TaskGroup tg-f52b1a9c-6051-4206-93b4-19dc1acbd505 made progress
Task tg-f52b1a9c-6051-4206-93b4-19dc1acbd505-tr-0: running
Task tg-f52b1a9c-6051-4206-93b4-19dc1acbd505-tr-0: done
Task tg-f52b1a9c-6051-4206-93b4-19dc1acbd505-tr-0: fetching
TaskGroup tg-f52b1a9c-6051-4206-93b4-19dc1acbd505 made progress
TaskSet ts-7bf2fb62-7915-499e-bca5-0c634a7c07c0: compleated
Getting results


,fileName,filePath,sizeInBytes
0,file_4.parquet,../_example_data/motor_colisions/file_4.parquet,12671778
1,file_2.parquet,../_example_data/motor_colisions/file_2.parquet,12689973
2,file_6.parquet,../_example_data/motor_colisions/file_6.parquet,13104800
3,file_3.parquet,../_example_data/motor_colisions/file_3.parquet,12631968
4,file_5.parquet,../_example_data/motor_colisions/file_5.parquet,12647973
5,file_8.parquet,../_example_data/motor_colisions/file_8.parquet,13173939
6,file_0.parquet,../_example_data/motor_colisions/file_0.parquet,12701131
7,file_9.parquet,../_example_data/motor_colisions/file_9.parquet,12460323
8,file_1.parquet,../_example_data/motor_colisions/file_1.parquet,12762999
9,file_7.parquet,../_example_data/motor_colisions/file_7.parquet,13098756


In [3]:
#session.fs.ls("/Volumes/data/")

In [4]:
silver_sql = """
    SELECT 
        strptime(CONCAT("CRASH DATE", ' - ', "CRASH TIME"), '%m/%d/%Y - %H:%M') as crash_datetime,
        "BOROUGH" as borough,
        "ZIP CODE" as zip_code,
        "LATITUDE" as latitude,
        "LONGITUDE" as longitude,
        "LOCATION" as location,
        "ON STREET NAME" as on_street_name,
        "CROSS STREET NAME" as cross_street_name,
        "OFF STREET NAME" as off_street_name,
        "NUMBER OF PERSONS INJURED" as number_of_persons_injured,
        "NUMBER OF PERSONS KILLED" as number_of_persons_killed,
        "NUMBER OF PEDESTRIANS INJURED" as number_of_pedestrians_injured,
        "NUMBER OF PEDESTRIANS KILLED" as number_of_pedestrians_killed,
        "NUMBER OF CYCLIST INJURED" as number_of_cyclist_injured,
        "NUMBER OF CYCLIST KILLED" as number_of_cyclist_killed,
        "NUMBER OF MOTORIST INJURED" as number_of_motorist_injured,
        "NUMBER OF MOTORIST KILLED" as number_of_motorist_killed,
        "CONTRIBUTING FACTOR VEHICLE 1" as contributing_factor_vehicle_1,
        "CONTRIBUTING FACTOR VEHICLE 2" as contributing_factor_vehicle_2,
        "CONTRIBUTING FACTOR VEHICLE 3" as contributing_factor_vehicle_3,
        "CONTRIBUTING FACTOR VEHICLE 4" as contributing_factor_vehicle_4,
        "CONTRIBUTING FACTOR VEHICLE 5" as contributing_factor_vehicle_5,
        "COLLISION_ID" as collision_id,
        "VEHICLE TYPE CODE 1" as vehicle_type_code_1,
        "VEHICLE TYPE CODE 2" as vehicle_type_code_2,
        "VEHICLE TYPE CODE 3" as vehicle_type_code_3,
        "VEHICLE TYPE CODE 4" as vehicle_type_code_4,
        "VEHICLE TYPE CODE 5" as vehicle_type_code_5
    --FROM read_parquet('/Volumes/data/motor_colisions/*.parquet')
    FROM read_parquet('../_example_data/motor_colisions/*.parquet')
"""

In [5]:
# Delta tables 
from deltalake import write_deltalake, DeltaTable
import pyarrow as pa
from harpy.quack import QuackContext

def fetch_arrow_sql(sql:str) -> pa.Table:
    with QuackContext() as ctx:
        q = ctx.sql(sql)
        return q.arrow()
    

def write_deltalake_from_pa(df:pa.Table, path: str, mode:str) -> None:
    write_deltalake(path, df, mode=mode)

ts = session.create_task_set()
ts.add_maps([MapTask(name="fetch-pa-table", fun=fetch_arrow_sql, args=[], kwargs={'sql': silver_sql})])
ts.add_transform(TransformTask(name="write-pa-table", fun=write_deltalake_from_pa, args=[], kwargs={'path': '../_example_data/motor_colisions_silver/', 'mode': 'overwrite'}))

result = ts.execute()
if result.success == False:
    print(result.results[0].std_err)

TaskSet ts-59b508cb-4486-425e-b2da-e21b6149fadf: running
TaskGroup tg-d3c59194-a462-45e8-bdab-b9ad54a2c970 made progress
TaskGroup tg-d3c59194-a462-45e8-bdab-b9ad54a2c970 made progress
Task tg-d3c59194-a462-45e8-bdab-b9ad54a2c970-tr-0: queued
TaskGroup tg-d3c59194-a462-45e8-bdab-b9ad54a2c970 made progress
Task tg-d3c59194-a462-45e8-bdab-b9ad54a2c970-tr-0: running
Task tg-d3c59194-a462-45e8-bdab-b9ad54a2c970-tr-0: done
Task tg-d3c59194-a462-45e8-bdab-b9ad54a2c970-tr-0: fetching
TaskGroup tg-d3c59194-a462-45e8-bdab-b9ad54a2c970 made progress
TaskGroup tg-cc3516c7-0bcd-4f2e-b5ca-230ab78e8248 made progress
TaskGroup tg-cc3516c7-0bcd-4f2e-b5ca-230ab78e8248 made progress
Task tg-cc3516c7-0bcd-4f2e-b5ca-230ab78e8248-tr-0: queued
TaskGroup tg-cc3516c7-0bcd-4f2e-b5ca-230ab78e8248 made progress
Task tg-cc3516c7-0bcd-4f2e-b5ca-230ab78e8248-tr-0: running


In [ ]:
from deltalake import DeltaTable, write_deltalake
from harpy.quack import QuackContext
import pandas as pd
import pyarrow as pa

# Repartition data into N partitions
N = 10
df_count = session.sql("SELECT COUNT(*) as count FROM read_csv('/Volumes/data/Motor_Vehicle_Collisions_-_Crashes.csv')")
total_count = df_count.iloc[0]['count']
partition_size = total_count // N
remainder = total_count % N

def repart_map(location: str, partition_size: int, index: int, remainder: int) -> None:
    offset = partition_size * index
    limit = partition_size + (1 if index < remainder else 0)
    with QuackContext() as q:
        q.sql("""
            COPY (
                SELECT * FROM read_csv('{0}', ALL_VARCHAR=True) LIMIT {1} OFFSET {2}
            ) TO '/Volumes/data/motor_colisions/file_{3}.parquet' (FORMAT PARQUET, ROW_GROUP_SIZE 1024, COMPRESSION SNAPPY)
        """.format(location, limit, offset, index))

ts = session.create_task_set()
ts.add_maps([MapTask("split", repart_map, args=[], kwargs={'location': '/Volumes/data/Motor_Vehicle_Collisions_-_Crashes.csv', 'partition_size': partition_size, 'index': i, 'remainder': remainder}) for i in range(N)])

results = ts.execute()

# Verify the output
output_counts = [session.sql("SELECT COUNT(*) as count FROM read_parquet('/Volumes/data/motor_colisions/file_{0}.parquet')".format(i)).iloc[0]['count'] for i in range(N)]
total_output_count = sum(output_counts)

print(f"Total input count: {total_count}")
print(f"Total output count: {total_output_count}")

In [ ]:
session.sql("SELECT * FROM read_parquet('/Volumes/data/motor_colisions/file_0.parquet')").head()

In [ ]:
session.sql("SELECT * FROM read_parquet('/Volumes/data/motor_colisions/file_1.parquet') LIM").head()

In [22]:
sql_exp = """
SELECT 
    strptime(CONCAT("CRASH DATE", ' - ', "CRASH TIME"), '%m/%d/%Y - %H:%M') as crash_datetime,
    "BOROUGH" as borough,
    "ZIP CODE" as zip_code,
    "LATITUDE" as latitude,
    "LONGITUDE" as longitude,
    "LOCATION" as location,
    "ON STREET NAME" as on_street_name,
    "CROSS STREET NAME" as cross_street_name,
    "OFF STREET NAME" as off_street_name,
    "NUMBER OF PERSONS INJURED" as number_of_persons_injured,
    "NUMBER OF PERSONS KILLED" as number_of_persons_killed,
    "NUMBER OF PEDESTRIANS INJURED" as number_of_pedestrians_injured,
    "NUMBER OF PEDESTRIANS KILLED" as number_of_pedestrians_killed,
    "NUMBER OF CYCLIST INJURED" as number_of_cyclist_injured,
    "NUMBER OF CYCLIST KILLED" as number_of_cyclist_killed,
    "NUMBER OF MOTORIST INJURED" as number_of_motorist_injured,
    "NUMBER OF MOTORIST KILLED" as number_of_motorist_killed,
    "CONTRIBUTING FACTOR VEHICLE 1" as contributing_factor_vehicle_1,
    "CONTRIBUTING FACTOR VEHICLE 2" as contributing_factor_vehicle_2,
    "CONTRIBUTING FACTOR VEHICLE 3" as contributing_factor_vehicle_3,
    "CONTRIBUTING FACTOR VEHICLE 4" as contributing_factor_vehicle_4,
    "CONTRIBUTING FACTOR VEHICLE 5" as contributing_factor_vehicle_5,
    "COLLISION_ID" as collision_id,
    "VEHICLE TYPE CODE 1" as vehicle_type_code_1,
    "VEHICLE TYPE CODE 2" as vehicle_type_code_2,
    "VEHICLE TYPE CODE 3" as vehicle_type_code_3,
    "VEHICLE TYPE CODE 4" as vehicle_type_code_4,
    "VEHICLE TYPE CODE 5" as vehicle_type_code_5
FROM read_parquet('../_example_data/motor_colisions/*.parquet')
"""

In [ ]:
from harpy.quack import QuackContext

with QuackContext() as q:
    query = q.sql(sql_exp)
    arrow_table = query.arrow(rows_per_batch=1000)

In [ ]:
len(arrow_table.to_batches())

In [ ]:
session.close()